# Import Library

In [2]:
import tensorflow as tf
import pandas as pd
import requests
import json
import base64
import random


In [3]:
df = pd.read_csv("data/personality_dataset.csv")
df.pop("Personality")  # Kolom target

columns = df.columns.values
rand = random.randint(0, len(df) - 1)

features = df.values[rand]

inputs = {key: value for key, value in zip(columns, features)}
inputs


{'Time_spent_Alone': 3.0,
 'Stage_fear': 'No',
 'Social_event_attendance': 2.0,
 'Going_outside': 3.0,
 'Drained_after_socializing': 'No',
 'Friends_circle_size': 10.0,
 'Post_frequency': 4.0}

In [4]:
def string_feature(value):
    return tf.train.Feature(
        bytes_list=tf.train.BytesList(value=[bytes(value, "utf-8")])
    )

def float_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def int_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


In [5]:
def prepare_json(inputs: dict):
    feature_spec = dict()

    for key, value in inputs.items():
        if isinstance(value, float):
            feature_spec[key] = float_feature(value)
        elif isinstance(value, int):
            feature_spec[key] = int_feature(value)
        elif isinstance(value, str):
            feature_spec[key] = string_feature(value)

    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()

    result = [
        {
            "examples": {
                "b64": base64.b64encode(example).decode()
            }
        }
    ]

    return json.dumps({
        "signature_name": "serving_default",
        "instances": result
    })


In [11]:
def make_predictions(inputs):
    json_data = prepare_json(inputs)

    endpoint = "https://mlops-submission-production.up.railway.app/v1/models/personality_model:predict"
    response = requests.post(endpoint, data=json_data)

    try:
        prediction = response.json()["predictions"][0]  # [0.959925294]

        # Ambil nilai float dari list
        value = prediction[0] if isinstance(prediction, list) else prediction

        label = "Extrovert" if value > 0.5 else "Introvert"
        return label

    except Exception as e:
        return f"Error: {e}\nResponse: {response.text}"


In [12]:
make_predictions(inputs)

'Extrovert'